In [ ]:
import pandas as pd
import numpy as np
import os

In [ ]:
pwd


In [ ]:
raw_data=pd.read_csv("../input/deep-learning-az-ann/Churn_Modelling.csv")
raw_data


In [ ]:
data=raw_data.copy()
data

In [ ]:
data.describe()

In [ ]:
data=data.drop(['RowNumber','CustomerId','Surname'],axis=1)
data


In [ ]:
data

In [ ]:
nas=data.isnull().sum()
nas


In [ ]:
data2=data.drop(['Exited'],axis=1)
data2

In [ ]:
data_with_dummies=pd.get_dummies(data2,drop_first=True)
data_with_dummies=data_with_dummies.values

In [ ]:
num_counters=int(np.sum(data['Exited']))
num_counters

In [ ]:
indices_to_remove=[]
zeros_counter=0
for i in range (data_with_dummies.shape[0]):
    if data['Exited'][i]==0:
        zeros_counter=zeros_counter+1
        if zeros_counter>num_counters:
            indices_to_remove.append(i)   
indices_to_remove

In [ ]:
x=np.delete(data_with_dummies,indices_to_remove, axis=0)
x.shape


In [ ]:
y_data=data['Exited']
y_data=y_data.values

In [ ]:
y=np.delete(y_data,indices_to_remove, axis=0)
y

In [ ]:
from sklearn.preprocessing import StandardScaler

In [ ]:
shuffled_indices=np.arange(x.shape[0])
np.random.shuffle(shuffled_indices)
shuffled_indices


In [ ]:
x_shuffled=x[shuffled_indices]
x_shuffled


In [ ]:
y_shuffled=y[shuffled_indices]
y_shuffled


In [ ]:
scaler=StandardScaler()
x_scaled=scaler.fit_transform(x_shuffled)
x_scaled

In [ ]:
train_counts=int(0.8*shuffled_indices.shape[0])
validation_counts=int(0.1*shuffled_indices.shape[0])
train_counts

In [ ]:
x_train=x_scaled[:train_counts]
x_train

In [ ]:
x_validation=x_scaled[train_counts:train_counts+validation_counts]
x_validation

In [ ]:
x_test=x_scaled[train_counts+validation_counts:]
x_test

In [ ]:
y_train=y_shuffled[:train_counts]
y_train.shape


In [ ]:
y_validation=y_shuffled[train_counts:train_counts+validation_counts]
y_validation.shape

In [ ]:
y_test=y_shuffled[train_counts+validation_counts:]
y_test.shape

In [ ]:
import tensorflow as tf

In [ ]:
input_size=11
output_size=2
hidden_size=50

In [ ]:
model=tf.keras.Sequential([
    
    tf.keras.layers.Dense(hidden_size,activation='relu'),
    tf.keras.layers.Dense(hidden_size,activation='relu'),
    tf.keras.layers.Dense(output_size,activation='softmax')
])

In [ ]:
model.compile(optimizer='adam',loss='sparse_categorical_crossentropy',metrics=['accuracy'])

In [ ]:
batch_size=100
max_epochs=100

In [ ]:
model.fit(x_train,y_train,batch_size=batch_size,epochs=max_epochs,validation_data=(x_validation,y_validation),verbose=2)

In [ ]:
test_loss,test_accuracy=model.evaluate(x_test,y_test)

In [ ]:
train_loss,train_accuracy=model.evaluate(x_train,y_train)

In [ ]:
y_pred=model.predict(x_test)
y_pred=y_pred[:,1]
y_pred=y_pred>0.5
y_pred.shape


In [ ]:
y_test=y_test>0.5
y_test

In [ ]:
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test, y_pred)
print(cm)